In [ ]:
!git clone https://github.com/f90/Wave-U-Net-Pytorch.git
!wget https://www.dropbox.com/s/r374hce896g4xlj/models.7z?dl=1 -O models.7z 
!pip install -r Wave-U-Net-Pytorch/requirements.txt -q

Cloning into 'Wave-U-Net-Pytorch'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 275 (delta 9), reused 20 (delta 7), pack-reused 241
Receiving objects: 100% (275/275), 20.61 MiB | 26.51 MiB/s, done.
Resolving deltas: 100% (146/146), done.
--2021-11-09 15:04:48--  https://www.dropbox.com/s/r374hce896g4xlj/models.7z?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/r374hce896g4xlj/models.7z [following]
--2021-11-09 15:04:49--  https://www.dropbox.com/s/dl/r374hce896g4xlj/models.7z
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca684fda69601e09ad1a3daabea.dl.dropboxusercontent.com/cd/0/get/BZodfGH8lDQ4aV2c9pa0OcKrWk

In [ ]:
!7za e models.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 255436762 bytes (244 MiB)

Extracting archive: models.7z
--
Path = models.7z
Type = 7z
Physical Size = 255436762
Headers Size = 223
Method = LZMA2:26
Solid = +
Blocks = 1

  0%      1% 1        2% 1 - waveunet/model                         4% 1 - waveunet/model                         5% 1 - waveunet/model                         6% 1 - waveunet/model                         8% 1 - waveunet/model                         9% 1 - waveunet/model   

In [1]:
!cp -R /content/drive/MyDrive/Wave-U-Net-Pytorch/ /content/
!cp /content/drive/MyDrive/model /content/

In [2]:
!pip install -r Wave-U-Net-Pytorch/requirements.txt -q

     |████████████████████████████████| 753.4 MB 7.2 kB/s 
     |████████████████████████████████| 4.0 MB 41.9 MB/s 
     |████████████████████████████████| 963 kB 69.0 MB/s 
     |████████████████████████████████| 129 kB 67.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.4.0 which is incompatible.


In [3]:
import pandas as pd
data = pd.read_json('/content/drive/MyDrive/song_lyric_map.json')

In [4]:
!pip install torchaudio -q

     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:39tcmalloc: large alloc 1147494400 bytes == 0x557542194000 @  0x7f618d64b615 0x557507dff4cc 0x557507edf47a 0x557507e022ed 0x557507ef3e1d 0x557507e75e99 0x557507e709ee 0x557507e03bda 0x557507e75d00 0x557507e709ee 0x557507e03bda 0x557507e72737 0x557507ef4c66 0x557507e71daf 0x557507ef4c66 0x557507e71daf 0x557507ef4c66 0x557507e71daf 0x557507e04039 0x557507e47409 0x557507e02c52 0x557507e75c25 0x557507e709ee 0x557507e03bda 0x557507e72737 0x557507e709ee 0x557507e03bda 0x557507e71915 0x557507e03afa 0x557507e71c0d 0x557507e709ee
     |████████████████████████████████| 881.9 MB 18 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.5.0 requires torch==1.4.0, but you have torch 1.10.0 which is incompatible.
torchtext 0.10.0 requires 

In [5]:
data = pd.read_json('/content/drive/MyDrive/mp3_data_genres.json')

In [6]:
data.dropna(subset=['Samputi'],inplace=True)
data['Offset'].fillna(60,inplace=True)

In [ ]:
import sys
sys.path.append('/content/Wave-U-Net-Pytorch/')

In [ ]:
import librosa

In [13]:
import os
import subprocess
import torch
import torchaudio
import torchaudio.transforms
from google.colab import files

SRC_AUD_PATH = '/content/drive/MyDrive/Annamayya/'
TMP_AUD_PATH = '/content/'
FREQ = 16000

def safe_delete(p):
  if os.path.exists(p): os.remove(p)

def get_mfcc_vocal(inps):
  t = inps[0]
  if os.path.exists(f'/content/drive/MyDrive/vocal_mfcc/{t}.pt'):
    return
  print('Processing ', t)
  off = inps[1]
  cmd = f'-i "{SRC_AUD_PATH}{t}" -acodec pcm_s16le -ac 1 -ar 16000 "{TMP_AUD_PATH}{t}.wav"'
  cmd2 = f'/content/Wave-U-Net-Pytorch/predict.py --cuda --load_model /content/model --input "{TMP_AUD_PATH}{t}.wav"'
  cmd3 = f'{TMP_AUD_PATH}{t}.wav'
  os.system(f"ffmpeg {cmd}")
  try:
    os.system(f"python {cmd2}")
  except:
    print('Error with ', t)
    return
  voc_path = f'{TMP_AUD_PATH}{t}.wav_vocals.wav'
  waveform, sample_rate = torchaudio.load(voc_path,
                                          num_frames=int(120*FREQ),
                                          frame_offset=int(off*FREQ))
  if len(waveform) == 0:
    print('Error with ', t)
    return
  try:
    mfcc_tx = torchaudio.transforms.MFCC(FREQ, n_mfcc=40)
    mfcc = mfcc_tx(waveform)
  except:
    print('Error with ', t)
    return
  torch.save(mfcc,f'/content/drive/MyDrive/vocal_mfcc/{t}.pt')
  os.remove(cmd3)
  os.remove(f'{TMP_AUD_PATH}{t}.wav_vocals.wav')
  os.remove(f'{TMP_AUD_PATH}{t}.wav_other.wav')
  os.remove(f'{TMP_AUD_PATH}{t}.wav_bass.wav')
  os.remove(f'{TMP_AUD_PATH}{t}.wav_drums.wav')

In [15]:
import multiprocessing as mp
from multiprocessing import Pool
args = []
tmpi = 0
for i in data.index:
  args.append((data.at[i,'MP3 File'].strip(),data.at[i,'Offset']))
  if len(args) == 3:
    with Pool(3) as p:
      p.map(get_mfcc_vocal,args)
    args = []
    tmpi = tmpi + 1
    if tmpi > 400: break

Processing  ANNAMAYYA SANKIRTANAS_170 ANNAMAYYA BALA GANAMRUTHAM - 6_12 BALUVAGU TANA.mp3


/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


Error with  ANNAMAYYA SANKIRTANAS_170 ANNAMAYYA BALA GANAMRUTHAM - 6_12 BALUVAGU TANA.mp3
Processing  ANNAMAYYA SANKIRTANAS_226 ANNAMAYYA SRI VENKATESWARA VIJAYAM_03 MANUSULAKU.mp3
Processing  ANNAMAYYA SANKIRTANAS_226 ANNAMAYYA SRI VENKATESWARA VIJAYAM_02 SRI VENKATESWARUDU.mp3
Processing  ANNAMAYYA SANKIRTANAS_226 ANNAMAYYA SRI VENKATESWARA VIJAYAM_04 SIGGARI PENDLI.mp3


/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "
/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "
/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


Processing  ANNAMAYYA SANKIRTANAS_226 ANNAMAYYA SRI VENKATESWARA VIJAYAM_06 ITTIDIVO SATI.mp3
Processing  ANNAMAYYA SANKIRTANAS_226 ANNAMAYYA SRI VENKATESWARA VIJAYAM_05 ENNATIKI NIKU.mp3
Processing  ANNAMAYYA SANKIRTANAS_226 ANNAMAYYA SRI VENKATESWARA VIJAYAM_07 VINARAMMA CELULALA.mp3


...

/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "
/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "
/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


Processing  ANNAMAYYA SANKIRTANAS_230 ANNAMAYYA PATAKU PATTABHISHEKAM-15_04 KAIKONNA KOLADI.mp3
Processing  ANNAMAYYA SANKIRTANAS_230 ANNAMAYYA PATAKU PATTABHISHEKAM-15_05 SAHAJA CARAMU.mp3
Processing  ANNAMAYYA SANKIRTANAS_230 ANNAMAYYA PATAKU PATTABHISHEKAM-15_06 SESINATTE VANI.mp3


/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "
/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "
/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


Processing  ANNAMAYYA SANKIRTANAS_230 ANNAMAYYA PATAKU PATTABHISHEKAM-15_07 SURALU SANTHOSHINCIRI.mp3
Processing  ANNAMAYYA SANKIRTANAS_230 ANNAMAYYA PATAKU PATTABHISHEKAM-15_08 TAPPADU I ARDHAMU.mp3
Processing  ANNAMAYYA SANKIRTANAS_230 ANNAMAYYA PATAKU PATTABHISHEKAM-15_09 ELA SIGGULU.mp3


/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "
/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "
/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "
